### 用正则匹配规则search

In [1]:
import regex as re

example1 = ['.{0,10}(王先生|你).{0,20}(外观|动力).{0,30} ', '.{0,25}(没关系|不用担心|别担心).{0,35}(试驾|是假|事假).{0,25}']

query1 = '这款车自从今年面市以来，几乎每天都有不少客户来试驾，您看我们的预约表每天都安排的满满的。'

In [2]:
lgf = '.{0,10}(王先生|你).{0,20}(外观|动力).{0,30}'

def read_lgf(lfg):
    items = [item for item in lgf.strip().split('.') if item]
    ans = []
    for item in items:
        result = find_item(item)
        ans.append(result)
    return ans
         
def find_item(item):
    pos, endpos = [eval(i) for i in item[item.find('{') + 1: item.find('}')].split(',') if i.isdigit()]
    keyword = ''
    pattern=None
    if item.find('(') != -1:
        keyword = item[item.find('(') + 1: item.find(')')]
    if keyword:
        pattern = re.compile(keyword)
    return {'pos': pos, 'endpos': endpos, 'keyword': keyword, 'pattern': pattern}

response = read_lgf(lgf)
print('response: ', response)

response:  [{'pos': 0, 'endpos': 10, 'keyword': '王先生|你', 'pattern': regex.Regex('王先生|你', flags=regex.V0)}, {'pos': 0, 'endpos': 20, 'keyword': '外观|动力', 'pattern': regex.Regex('外观|动力', flags=regex.V0)}, {'pos': 0, 'endpos': 30, 'keyword': '', 'pattern': None}]


In [4]:
def match(text):
    pass
    

In [ ]:
re.findall?

Signature:
re.findall(
    pattern,
    string,
    flags=0,
    pos=None,
    endpos=None,
    overlapped=False,
    concurrent=None,
    timeout=None,
    ignore_unused=False,
    **kwargs,
)
Docstring:
Return a list of all matches in the string. The matches may be overlapped
if overlapped is True. If one or more groups are present in the pattern,
return a list of groups; this will be a list of tuples if the pattern has
more than one group. Empty matches are included in the result.
File:      /data/opts/anaconda3/lib/python3.8/site-packages/regex/regex.py
Type:      function
